In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests


In [2]:
!apt install build-essential libpoppler-cpp-dev pkg-config

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
libpoppler-cpp-dev is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [3]:
!pip install pdftotext

In [4]:
import pdftotext

In [5]:
%%time
text = []
with open('/content/PythonNotesForProfessionals.pdf', 'rb')  as pdfile:
    pdf = pdftotext.PDF(pdfile)
    for pge in pdf:
      text.append(pge)

CPU times: user 8.61 s, sys: 20 ms, total: 8.63 s
Wall time: 8.82 s


In [6]:
text = " ".join(text[28:841])

In [7]:
tokens = text.split()

In [8]:
len(set(tokens)), len(tokens)

(30312, 183835)

In [9]:
length = 20 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 115000:
    break


In [10]:
lines[0]

'You can use the Python 3 print function in Python 2 with the following import statement: from __future__ import print_function Python'

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer = Tokenizer(filters="",lower=False)

In [13]:
tokenizer.fit_on_texts(lines)
seq = tokenizer.texts_to_sequences(lines)

In [14]:
seq = np.array(seq)
X, y = seq[:, :-1], seq[:,-1]


In [15]:
vocab_size = len(tokenizer.word_index) + 1

In [16]:
y = to_categorical(y,num_classes=vocab_size)

In [17]:
seq_length = X.shape[1]
seq_length

20

In [22]:
import os
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            1003000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 20060)             2026060   
Total params: 3,179,960
Trainable params: 3,179,960
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [26]:
model.fit(X, y, batch_size = 256, epochs = 100,callbacks=[cp_callback])

Epoch 1/100
450/450 [==============================] - 18s 34ms/step - loss: 8.2387 - accuracy: 0.0337

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/100
450/450 [==============================] - 15s 34ms/step - loss: 7.4374 - accuracy: 0.0352

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/100
450/450 [==============================] - 15s 34ms/step - loss: 7.1529 - accuracy: 0.0618

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/100
450/450 [==============================] - 15s 34ms/step - loss: 6.7755 - accuracy: 0.0860

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/100
450/450 [==============================] - 15s 34ms/step - loss: 6.4245 - accuracy: 0.1056

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/100
450/450 [==============================] - 15s 34ms/step - loss: 6.1391 - accuracy: 0.1238

Epoch 00006: saving model to training_1/cp.ckpt
Epoch 7/100
450/450 [==============================] - 15s 34ms/step - loss: 5.8862 - ac

In [27]:
seed_text=lines[12343]
seed_text,lines[12344]

('False 4 not in {1,2,3} # True # Add and Remove s = {1,2,3} s.add(4) # s == {1,2,3,4} s.discard(3) #',
 '4 not in {1,2,3} # True # Add and Remove s = {1,2,3} s.add(4) # s == {1,2,3,4} s.discard(3) # s')

In [33]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return seed_text + ' '.join(text)

In [34]:
generate_text_seq(model, tokenizer, seq_length, "Python is not a interpredted", 20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


"Python is not a interpredted library, and prompt, so, daylight savings data s[0] # Spawn S.' Python line (combination of N,NE,E,SE,S,SW,W,NW) The module foobar.py: classlibrary, and prompt, so, daylight savings data s[0] # Spawn S.' Python line (combination of N,NE,E,SE,S,SW,W,NW) The module foobar.py: class"